In [1]:
# Reference https://www.youtube.com/watch?v=kCc8FmEb1nY
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [31]:
# hyperparameters
batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
assert device=='cuda', "error - this script requires GPU"
eval_iters=00

In [2]:
# read data
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
print(text[:50])

First Citizen:
Before we proceed any further, hear


In [5]:
# unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]  # encoder string to integers
decode = lambda l: ''.join([itos[i] for i in l])  # decoder integers to string

print(encode("hi there"))
print(decode(encode("hi there")))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [7]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)

In [8]:
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
split = int(len(data) * 0.9)
train_data = data[:split]
val_data = data[split:]

In [10]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
print("x: ", x)
print("y: ", y)
for n in range(block_size):
    target = y[n]
    context = x[:n+1]

    print("with target {} context {}".format(target, context))

x:  tensor([18, 47, 56, 57, 58,  1, 15, 47])
y:  tensor([47, 56, 57, 58,  1, 15, 47, 58])
with target 47 context tensor([18])
with target 56 context tensor([18, 47])
with target 57 context tensor([18, 47, 56])
with target 58 context tensor([18, 47, 56, 57])
with target 1 context tensor([18, 47, 56, 57, 58])
with target 15 context tensor([18, 47, 56, 57, 58,  1])
with target 47 context tensor([18, 47, 56, 57, 58,  1, 15])
with target 58 context tensor([18, 47, 56, 57, 58,  1, 15, 47])


In [ ]:
torch.manual_seed(1234)
batch_size = 4 # number of samples / batch
block_size = 8 # max context for prediction

In [11]:
def get_batch(split):
    # grab a batch of data x,y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

In [12]:
xb, yb = get_batch('train')
print('inputs:\n', xb)
print('targets:\n', yb)

inputs:
 tensor([[51, 59, 57, 58,  1, 39, 61, 39],
        [63,  6,  1, 61, 46, 53,  5, 57],
        [39,  1, 41, 46, 47, 50, 42, 10],
        [53, 59, 10,  0, 37, 53, 59,  1]])
targets:
 tensor([[59, 57, 58,  1, 39, 61, 39, 63],
        [ 6,  1, 61, 46, 53,  5, 57,  1],
        [ 1, 41, 46, 47, 50, 42, 10,  0],
        [59, 10,  0, 37, 53, 59,  1, 57]])


In [13]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when  the target: {target} input is {context.tolist()}")

when  the target: 59 input is [51]
when  the target: 57 input is [51, 59]
when  the target: 58 input is [51, 59, 57]
when  the target: 1 input is [51, 59, 57, 58]
when  the target: 39 input is [51, 59, 57, 58, 1]
when  the target: 61 input is [51, 59, 57, 58, 1, 39]
when  the target: 39 input is [51, 59, 57, 58, 1, 39, 61]
when  the target: 63 input is [51, 59, 57, 58, 1, 39, 61, 39]
when  the target: 6 input is [63]
when  the target: 1 input is [63, 6]
when  the target: 61 input is [63, 6, 1]
when  the target: 46 input is [63, 6, 1, 61]
when  the target: 53 input is [63, 6, 1, 61, 46]
when  the target: 5 input is [63, 6, 1, 61, 46, 53]
when  the target: 57 input is [63, 6, 1, 61, 46, 53, 5]
when  the target: 1 input is [63, 6, 1, 61, 46, 53, 5, 57]
when  the target: 1 input is [39]
when  the target: 41 input is [39, 1]
when  the target: 46 input is [39, 1, 41]
when  the target: 47 input is [39, 1, 41, 46]
when  the target: 50 input is [39, 1, 41, 46, 47]
when  the target: 42 input is 

In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off thelogits from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        if targets is None:
            loss = None
        else:
            B, T, C, = logits.shape         
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):

        # idx is (B, T) array of indices in the current context
        for n in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            #apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, c)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            idx = torch.cat((idx, idx_next), dim=1) #B, T+1)

        return idx
    

In [15]:
# x only
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb)
print(f"logits.shape {logits.shape}")
print(f"loss {loss}")

logits.shape torch.Size([4, 8, 65])
loss None


In [16]:
# x and y
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(f"logits.shape {logits.shape}")
print(f"loss {loss}")

logits.shape torch.Size([32, 65])
loss 4.952663898468018


In [25]:
m = BigramLanguageModel(vocab_size)
idx = torch.ones((4,1), dtype=torch.long)
nextchars = m.generate(idx, max_new_tokens=300)
nextchar = nextchars[0].tolist()
decode(nextchar)

" sEPsGuvG?lSY3yzivSyYQnOnXucUYnNRT:lFOe; \nN.-bpZIEEP&ANjUialFkVC.vaE3enOjqM:KdXA D:aumA$Ehr L.v$$uiGQQxwsKCCKdLKKBsiJq$VB 'hDq\ndF:ODKeYtm:-eCw VG\nZZsHPnDOp-RVNqvGRh'Ip bTxIEzIElpPPlo&:-oknw bCgPw.VIblvaLdYio3W UQ3mXclDPF\n?N$FFyGn;Lge$PUTJWEC;ZY.yye$-uptZINYQWU&B\nK3R$Pnj;Qa3!lFyZcx:bYuVvZ?N.YmNT:M:icL"

In [26]:
# create optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [27]:
batch_size = 32 # number of samples / batch

for steps in range(10000):

    # sample batch of data
    xb, yb = get_batch('train')

    # evaluate loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.4813098907470703


In [29]:
idx = torch.ones((4,1), dtype=torch.long)
nextchars = m.generate(idx, max_new_tokens=400)
nextchar = nextchars[0].tolist()
decode(nextchar)

" is, asher-qPoothan:\nF Cl.\nETevear paug CI nd?\nEThas thtewoy femprd at ar.\n\nWhwho, wr se, aleldouled brne, ETor R:\nThasprs h nd a l ie.\n\nPEd tou alou uit!\nPAPAn lled h OMy 'e ikiee w bs shes, nce? mesthankir ar no thit de, gr tinough\nI cthit w,\n\nDUMais,\nShe cthodoreth,\nYoeieen we ty, mat averck?\nDubanverivend,\nIO:\nNE lvGof br nctallrsir sat merm. n? I thard wihingse te sesthyod ontiustre-sthe y ili"